# Spotify API Scrape

In [ ]:
# run this once
#!pip install spotipy --upgrade

In [74]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import json
import os

In [3]:
with open("../api_cred.json") as json_file:
    creds = json.load(json_file)

In [6]:
client_id = creds['client_id']
client_secret = creds['client_secret']
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [7]:
# api call
resp = sp.search(q="Korean Pop", type="playlist")

In [60]:
playlist_ids = []
for playlist in resp['playlists']['items']:
    playlist_ids.append(playlist['id'])

In [63]:
playlists_data = []
for playlist_id in playlist_ids:
    # api call
    playlist_data = sp.playlist(playlist_id)
    playlists_data.append(playlist_data)

In [68]:
kpop_df = pd.DataFrame()
for cur_playlist_data in playlists_data[0:2]: # only doing two for api reasons currently
    cur_playlist_track_ids = []
    cur_playlist_song_names = []
    # can add more features here (album, artist, etc..)
    for track in cur_playlist_data['tracks']['items']:
        cur_playlist_track_ids.append(track['track']['id'])
        cur_playlist_song_names.append(track['track']['name'])
    # api call
    playlist_track_feats = sp.audio_features(cur_playlist_track_ids)
    for i in range(len(playlist_track_feats)):
        playlist_track_feats[i]['song_name'] = cur_playlist_song_names[i]
    cur_playlist_df = pd.DataFrame(playlist_track_feats)
    kpop_df = pd.concat([kpop_df, cur_playlist_df])

In [77]:
kpop_df = kpop_df.reset_index(drop=True)
csv_path = "../data/test/kpop/metadata/"
if not os.path.exists(csv_path):
    os.makedirs(csv_path)
kpop_df.to_csv("../data/test/kpop/metadata/kpop_metadata.csv",index=False)

In [79]:
analysis_df = pd.read_csv("../data/test/kpop/metadata/kpop_metadata.csv")